In [1]:
"""
DATASET : https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/
"""

'\nDATASET : https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/\n'

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
dataset = pd.read_csv('creditcard.csv')
X = StandardScaler().fit_transform(dataset.drop('Class', axis=1))
y = dataset['Class']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Build the Autoencoder model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

input_dim, encoding_dim = X_train.shape[1], 16
encoder = Sequential([
    InputLayer(input_shape=(input_dim,)),
    Dense(32, activation='relu'),
    Dense(encoding_dim, activation='relu')
])

In [6]:
decoder = Sequential([
    InputLayer(input_shape=(encoding_dim,)),
    Dense(32, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

In [7]:
autoencoder = Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

autoencoder.summary()

Epoch 1/10
7121/7121 [==============================] - 15s 2ms/step - loss: 0.7737 - val_loss: 0.7232
Epoch 2/10
7121/7121 [==============================] - 18s 2ms/step - loss: 0.7275 - val_loss: 0.7092
Epoch 3/10
7121/7121 [==============================] - 16s 2ms/step - loss: 0.7184 - val_loss: 0.7033
Epoch 4/10
7121/7121 [==============================] - 16s 2ms/step - loss: 0.7150 - val_loss: 0.7012
Epoch 5/10
7121/7121 [==============================] - 17s 2ms/step - loss: 0.7135 - val_loss: 0.7007
Epoch 6/10
7121/7121 [==============================] - 16s 2ms/step - loss: 0.7125 - val_loss: 0.7001
Epoch 7/10
7121/7121 [==============================] - 15s 2ms/step - loss: 0.7116 - val_loss: 0.6983
Epoch 8/10
7121/7121 [==============================] - 17s 2ms/step - loss: 0.7107 - val_loss: 0.6982
Epoch 9/10
7121/7121 [==============================] - 18s 3ms/step - loss: 0.7099 - val_loss: 0.6966
Epoch 10/10
7121/7121 [==============================] - 17s 2ms/step - l

In [8]:
# Evaluate the model
predictions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
threshold = np.percentile(mse, 95)

# Classify anomalies
y_pred = np.where(mse > threshold, 1, 0)

1781/1781 [==============================] - 4s 2ms/step


In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
print(f'Classification Report:\n{classification_report(y_test, y_pred)}')

Accuracy: 0.951353533934904
Confusion Matrix:
[[54103  2761]
 [   10    88]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     56864
           1       0.03      0.90      0.06        98

    accuracy                           0.95     56962
   macro avg       0.52      0.92      0.52     56962
weighted avg       1.00      0.95      0.97     56962

